In [2]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


## Select Object Indices of Interest

| Object Indices |[]()|[]()|
|---|---|---|
| __1.__ 002_master_chef_can | __8.__ 009_gelatin_box      | __15.__ 035_power_drill       |
| __2.__ 003_cracker_box     | __9.__ 010_potted_meat_can  | __16.__ 036_wood_block        |
| __3.__ 004_sugar_box       | __10.__ 011_banana          | __17.__ 037_scissors          |
| __4.__ 005_tomato_soup_can | __11.__ 019_pitcher_base    | __18.__ 040_large_marker      |
| __5.__ 006_mustard_bottle  | __12.__ 021_bleach_cleanser | __19.__ 051_large_clamp       |
| __6.__ 007_tuna_fish_can   | __13.__ 024_bowl            | __20.__ 052_extra_large_clamp |
| __7.__ 008_pudding_box     | __14.__ 025_mug             | __21.__ 061_foam_brick        |

In [3]:
from transforms3d.quaternions import quat2mat, mat2quat

def getPoseCNNQuat(data, obj):
    pose_idx = np.where(data['rois'][:,1].flatten()==obj)[0]
    if(len(pose_idx) == 0):
        return None, None
    else:
        pose_idx = pose_idx[0]
    pose = data['poses'][pose_idx]
    q = pose[:4][[1,2,3,0]]
    q /= np.linalg.norm(q)
    t = pose[4:7]
    return q, t

## YCB Dataset

In [20]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes
from object_pose_utils.datasets.feature_dataset import  FeatureDataset
dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

feature_root = '/scratch/bokorn/results/posecnn_feat_all/'
feature_key = 'fc6'
feature_size = 4096

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.TRANSFORM_MATRIX,
                ]

ycb_dataset = YCBDataset(dataset_root, mode=mode,
                     object_list = object_list,
                     output_data = output_format,
                     resample_on_error = False,
                     add_syn_background = False,
                     add_syn_noise = False,
                     #use_posecnn_data = True,
                     #preprocessors = [InplaneRotator()],
                     #postprocessors = [ImageNormalizer()],
                     image_size = [640, 480], num_points=1000)

model_clouds = {}
for object_id in object_list:
    cloud_filename = '{}/models/{}/points.xyz'.format(dataset_root, dataset.classes[object_id])
    model_clouds[object_id] = np.loadtxt(cloud_filename)

In [19]:
print(len(dataset[0]))
print(len(ycb_dataset))

Exception on index 0: [Errno 2] No such file or directory: '/scratch/bokorn/results/posecnn_feat_all/data/0048/000001_002_master_chef_can_feat.npz'
4
14025


In [24]:
from se3_distributions.datasets.ycb_dataset import getYCBSymmeties
from object_pose_utils.utils.pose_processing import symmetricAngularDistance
from object_pose_utils.utils.pose_error import add, adi
from quat_math import quaternion_matrix

import pathlib

from tqdm import tqdm_notebook as tqdm

pose_gt = {obj:{} for obj in object_list}
pose_est = {obj:{} for obj in object_list}
sym_angular_error = {obj:{} for obj in object_list}
add_error = {obj:{} for obj in object_list}
add_sym_error = {obj:{} for obj in object_list}

bad_data = []
with torch.no_grad():
    for j, data in enumerate(tqdm(ycb_dataset)):
        obj, quat, trans, _ = data
        posecnn_path = '{}/data/{}-posecnn.mat'.format(dataset.dataset_root, dataset.getPath(j))
        posecnn_data = scio.loadmat(posecnn_path)
        q_est, t_est = getPoseCNNQuat(posecnn_data, obj.item())
        
        if(len(obj) == 0):
            bad_data.append(j)
            continue

        if(quat is None or q_est is None):
            bad_data.append(j)
            continue
        res = (feat, torch.tensor(pred_q).float(), pred_t)

        sym_axis, sym_ang = getYCBSymmeties(obj.item())
        trans = to_np(trans)

        mat = quaternion_matrix(quat)

        err_ang = symmetricAngularDistance(torch.Tensor(q_est).unsqueeze(0), 
                                           quat.unsqueeze(0),
                                           sym_axis, sym_ang).item()*180/np.pi
        mat_est = quaternion_matrix(q_est)
        err_add = add(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                      model_clouds[obj.item()])
        err_adi = adi(mat[:3,:3], trans, mat_est[:3,:3], t_est, 
                      model_clouds[obj.item()])

        pose_est[obj.item()][j]=(q_est, t_est)
        pose_gt[obj.item()][j]=(to_np(quat), trans)
        sym_angular_error[obj.item()][j]=err_ang
        add_error[obj.item()][j]=err_add
        add_sym_error[obj.item()][j]=err_adi
            
np.savez('iros_results/posecnn.npz', 
         pose_gt=pose_gt,
         pose_est=pose_est,
         sym_angular_error=sym_angular_error,
         add_error=add_error,
         add_sym_error=add_sym_error,
         bad_data=bad_data
        )

In [27]:
ls iros_results/

dense_fusion.npz             hist_comp_global_gt_mask.npz  posecnn.npz
hist_comp_global.npz         hist_comp_global_prop.npz
hist_comp_global_funnel.npz  hist_comp_local.npz
